In [7]:
from unified_planning.shortcuts import *
from unified_planning.plans import *
from importlib import reload

In [8]:
import unifiedplanning_blocksworld as bw
import prompts
import planbench as pb
import policy_model as policy
import llm_utils
import data_loader
import pandas as pd
import regex as re

In [22]:
reload(policy)
reload(prompts)
reload(bw)
reload(pb)
reload(llm_utils)
reload(data_loader)

<module 'data_loader' from '/Users/akankshachawla/Desktop/WS-24-25/Thesis/planning-with-llms/src/data_loader.py'>

In [10]:
problem = bw.BlocksworldProblem()

In [81]:
pb.parse_planbench_initial_condition(problem, prompts.nextaction_problem_init)
pb.parse_planbench_goal_state(problem, prompts.nextaction_problem_goal)
print(problem.initial_values)
#initial vals ko vaapais readable mein translate karna hoga

{clear(blue): true, clear(yellow): true, hand_empty: true, on(blue, orange): true, on(orange, red): true, on(blue, blue): false, on(yellow, blue): false, on(orange, blue): false, on(red, blue): false, on(blue, yellow): false, on(yellow, yellow): false, on(orange, yellow): false, on(red, yellow): false, on(yellow, orange): false, on(orange, orange): false, on(red, orange): false, on(blue, red): false, on(yellow, red): false, on(red, red): false, clear(orange): false, clear(red): false, holding(blue): false, holding(yellow): false, holding(orange): false, holding(red): false}


In [11]:

pi = policy.PolicyModel()
    # def next_action_one_shot(self, 
    #                          problem_init, problem_goal,problem_action_hist,  # the state S
    #                          example_init, example_goal,example_action_hist,example_next_action)
plan=[]
action_tuple = pi.next_action_one_shot(prompts.nextaction_problem_init, prompts.nextaction_problem_goal,
                                       plan,
                                       prompts.nextaction_example_init, prompts.nextaction_example_goal,
                                       prompts.nextaction_example_action_hist,prompts.nextaction_example_nextaction)



RESPONSE:
 Let's analyze the initial state and the goal state.

Initial state:
- The blue block is clear
- The yellow block is clear
- The hand is empty
- The blue block is on top of the orange block
- The orange block is on top of the red block
- The red block is on the table
- The yellow block is on the table

Goal state:
- The red block is on top of the orange block
- The blue block is on top of the yellow block

To achieve the goal state, I suggest the following next action:

[NEXT ACTION] unstack the blue block from on top of the orange block [END NEXT ACTION]

This action will make the blue block clear and available to be stacked on top of the yellow block later.


In [82]:
action_tuple=('unstack','blue','orange')
print(action_tuple)
next_action=problem.create_action(action=action_tuple[0],blocks=action_tuple[1:])
print(next_action)

('unstack', 'blue', 'orange')
('blue', 'orange')
unstack(blue, orange)


In [83]:
simulation=problem.create_seq_simulation()
current_state=simulation.get_initial_state()
new_state=problem.validate_action(current_state,next_action,simulation)
if(new_state):
    print(new_state)
else:
    print('Invalid action proposed by policy')

{on(blue, orange): false, holding(blue): true, clear(orange): true, hand_empty: false, clear(blue): true, clear(yellow): true, on(orange, red): true, on(blue, blue): false, on(yellow, blue): false, on(orange, blue): false, on(red, blue): false, on(blue, yellow): false, on(yellow, yellow): false, on(orange, yellow): false, on(red, yellow): false, on(yellow, orange): false, on(orange, orange): false, on(red, orange): false, on(blue, red): false, on(yellow, red): false, on(red, red): false, clear(red): false, holding(yellow): false, holding(orange): false, holding(red): false}


In [85]:
#print(problem.goals)
list_goals=[]
for goal in problem.goals:
    list_goals.append(str(goal))
list_goals

['on(red, orange)', 'on(blue, yellow)']

In [86]:
dir(new_state)

['MAX_ANCESTORS',
 '__abstractmethods__',
 '__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_ancestors',
 '_condense_state',
 '_father',
 '_hash',
 '_values',
 'get_value',
 'make_child']

In [88]:
print(list(new_state._values))
state_values=list(new_state._values)
# for fluent in problem.fluents:
#     value = new_state.get_value(fluent)
#     print(f"{fluent}: {value}")


[on(blue, orange), holding(blue), clear(orange), hand_empty, clear(blue), clear(yellow), on(orange, red), on(blue, blue), on(yellow, blue), on(orange, blue), on(red, blue), on(blue, yellow), on(yellow, yellow), on(orange, yellow), on(red, yellow), on(yellow, orange), on(orange, orange), on(red, orange), on(blue, red), on(yellow, red), on(red, red), clear(red), holding(yellow), holding(orange), holding(red)]


In [129]:
for element in state_values:
    #print(str(element),'\n')
    string_f=str(element)
    if string_f in list_goals:
        print(string_f)

on(blue, yellow)
on(red, orange)


In [6]:
reload(data_loader)
df=data_loader.load_data("../data/train.parquet")
df.head()


,task,prompt_type,domain,instance_id,example_instance_ids,query,ground_truth_plan
885,task_1_plan_generation,oneshot,blocksworld_3,2,[1],I am playing with a set of blocks where I need...,(pick-up red)\n(stack red orange)\n(pick-up bl...
886,task_1_plan_generation,oneshot,blocksworld_3,3,[2],I am playing with a set of blocks where I need...,(unstack blue orange)\n(put-down blue)\n(unsta...
887,task_1_plan_generation,oneshot,blocksworld_3,4,[3],I am playing with a set of blocks where I need...,(unstack red blue)\n(put-down red)\n(unstack b...
888,task_1_plan_generation,oneshot,blocksworld_3,5,[4],I am playing with a set of blocks where I need...,(unstack red blue)\n(put-down red)\n(unstack b...
889,task_1_plan_generation,oneshot,blocksworld_3,6,[5],I am playing with a set of blocks where I need...,(unstack red orange)\n(put-down red)\n(unstack...


In [156]:
df.size

4200

In [161]:
reload(prompts)
prompt=prompts.make_prompt(df.iloc[0]['query'],type='full')
print(prompt)

I am playing with a set of blocks where I need to arrange the blocks into stacks
        Here are the actions I can do: Pick up a block, Unstack a block from on top of another block, Put down a block, Stack a block on top of another block."
        I have the following restrictions on my actions:
        I can only pick up or unstack one block at a time
        I can only pick up or unstack a block if my hand is empty
        I can only pick up a block if the block is on the table and the block is clear
        A block is clear if the block has no other blocks on top of it and if the block is not picked up
        I can only unstack a block from on top of another block if the block I am unstacking was really on top of the other block
        I can only unstack a block from on top of another block if the block I am unstacking is clear
        Once I pick up or unstack a block, I am holding the block
        I can only put down a block that I am holding
        I can only stack a block o

In [7]:
reload(pb)
pbstring=prompts.get_pbstring(df.iloc[0]['query'])
# matches = [m.start() for m in re.finditer(r'\[STATEMENT\]', pbstring)]
# if len(matches) >= 2:
#     problem_string = pbstring[matches[1]:]  # Extract from the second occurrence onward
#     print(problem_string)

init,goal=pb.parse_problem(pbstring)

#print(problem_string)

In [8]:
init

' the red block is clear, the blue block is clear, the orange block is clear, the hand is empty, the red block is on the table, the blue block is on the table and the orange block is on the table'

In [9]:
goal

' the red block is on top of the orange block and the blue block is on top of the red block'

In [175]:
conditions=problem_string.split("As initial conditions I have that,")[1]
goal_part=

 the red block is on top of the orange block and the blue block is on top of the red block


In [ ]:
def solve(pi,prompt,problem):
    s0 = problem.
    goal_reached = False
    i = 0
    max_iter = 10
    while not(goal_reached) and i<max_iter:
        action = pi.next_action_one_shot( ... ) 
        # is action possible?
        plan.append(action)
        s_new = s.apply(action)
        # check for completion, abort if completed
        score = r.get_reward(s_new)
        # later: use reward to choose among candidates
    return plan
#maintain action history
#pass this history and get model to act on it 
#then check possibiliy of action

In [11]:
#vanilla method
def main_fullSol(df):
    results=[]
    for i in range(0,1):
        print(f'\n\n--------------------Entering main loop PROBLEM #{i}--------------------')
        
        #declare Blocksworld Problem object
        problem = bw.BlocksworldProblem()

        #create prompt using PLANBENCH dataset
        #prompt=prompts.make_prompt(df.iloc[i]['query'])
        #print(f'\n\nMake prompt:{prompt} \nEND Make prompt')

        #create prompt using OUR dataset
        prompt,init,goal=prompts.make_prompt(df.iloc[i]['init'],df.iloc[i]['goal'],df.iloc[i]['demo_init'],df.iloc[i]['demo_goal'],df.iloc[i]['demo_plan'])

        #parse dataset into Blocksworld object
        #pbstring=prompts.get_pbstring(df.iloc[i]['query'])
        #init,goal=pb.parse_problem(pbstring)
        #pb.parse_planbench_initial_condition(problem, init)
        #pb.parse_planbench_goal_state(problem, goal)
        #print(f'\n\nBlocksworld Problem Initial Values:{problem.initial_values}')
        #print(f'\nBlocksworld Problem Goal Condition:{problem.goals}')
        
        #changed generation of Blocksworld object for OUR dataset
        pb.parse_planbench_initial_condition(problem, init)
        pb.parse_planbench_goal_state(problem, goal)
        print(f'\n\nBlocksworld Problem Initial Values:{problem.initial_values}')
        print(f'\nBlocksworld Problem Goal Condition:{problem.goals}')

        #activate solve mode
        pi = policy.PolicyModel()
        #query the llm, parse the actions, return tuples
        actions=pi.Vanilla_fullSol_one_shot(prompt)
        if not actions:
            print(f'\n\n LLM exceeded 3 tries')
            results.append(False)
            continue
        print(f'\n\nProposed actions by Policy : {actions} \n END Policy')
        model_plan=problem.create_plan_from_tuples(actions)

        #validate and apply    
        simulation=problem.create_seq_simulation()
        valid_state=problem.check_and_apply(simulation,model_plan)
        if not valid_state:
            results.append(False)
            continue
        
        #termination-check 
        if problem.terminate(valid_state):
            results.append(True)
            continue
        
        results.append(False)
    return results

In [23]:
#load data for PLANBENCH dataset
#df=data_loader.load_data("../data/train.parquet")
#df.head()
#results=main_fullSol(df)

In [25]:
#print(f"\n\n Model solution's verdict: {results}")

In [24]:
#df.head

In [18]:
#load data for OUR dataset
import pandas as pd
n=3
df=pd.read_csv(f'../data/Oneshot_val_{n}_blocks')
df.drop(columns='Unnamed: 0',inplace=True)

In [19]:
df.shape

(15, 5)

In [20]:
results=main_fullSol(df)



--------------------Entering main loop PROBLEM #0--------------------
State in create_ic (('violet', 'teal', 'brown'),)
State in create_goal (('violet',), ('teal',), ('brown',))
Blocks being handled:['brown']
Blocks being handled:[]
Blocks being handled:['violet']
Blocks being handled:['teal', 'violet']
Blocks being handled:['brown', 'teal']


Blocksworld Problem Initial Values:{clear(brown): true, hand_empty: true, on(teal, violet): true, on(brown, teal): true, on(brown, brown): false, on(violet, brown): false, on(teal, brown): false, on(brown, violet): false, on(violet, violet): false, on(violet, teal): false, on(teal, teal): false, clear(violet): false, clear(teal): false, holding(brown): false, holding(violet): false, holding(teal): false}

Blocksworld Problem Goal Condition:[clear(violet), clear(teal), clear(brown)]


.......Querying LLM for a plan......... iteration: #0


LLM RESPONSE: It looks like you want to write a plan to achieve the goal, just like you did in the first ex

In [21]:
print(f"\n\n Model solution's verdict: {results}")



 Model solution's verdict: [True]
